In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime as dt

from deep_hedging import Currency, Frequency, ConstantRateCurve
from deep_hedging.linear.commodity_bond import CommodityBond

## Input parameters.

In [3]:
RF_RATE = 0.15
Z_SPREAD = 0.05
CONVENIENCE_YIELD = 0.02

TIME_TILL_MATURITY = 2.0
FREQUENCY = Frequency.ANNUALLY

## Create curves.

In [4]:
yield_curve = ConstantRateCurve(
    currency="RUB",
    constant_rate=RF_RATE + Z_SPREAD,
    compounding_frequency=FREQUENCY,
)
forward_curve = ConstantRateCurve(
    currency="RUB",
    constant_rate=RF_RATE - CONVENIENCE_YIELD,
    compounding_frequency=Frequency.CONTINUOUS
)

## Create bond.

In [5]:
start = dt.datetime.today()
end = start + dt.timedelta(days=int(round(TIME_TILL_MATURITY * 365)))

gold_bond = CommodityBond(
    yield_curve=yield_curve,
    start_date=start,
    end_date=end,
    frequency=FREQUENCY,
    yield_curve_commodity=forward_curve,
)
gold_bond

StructuredNote of:
1. LONG 0.949 units of ZeroCouponBond:
* CCY = Currency.RUB
* Term = 2.0 years
* YTM = 20.0%
* Start Date = 2024-07-22 19:44:14.625424
* End Date = 2026-07-22 19:44:14.625424.

2. LONG 1.0537 units of Forward:
* Term = 2.0 years
* Strike = 100.0%
* Start Date = 2024-07-22 19:44:14.625424
* End Date = 2026-07-22 19:44:14.625424.

3. LONG 0.051 units of ZeroCouponBond:
* CCY = Currency.RUB
* Term = 1.0 years
* YTM = 20.0%
* Start Date = 2024-07-22 19:44:14.625424
* End Date = 2025-07-23 00:00:00.

4. LONG 0.0537 units of Forward:
* Term = 1.0 years
* Strike = 100.0%
* Start Date = 2024-07-22 19:44:14.625424
* End Date = 2025-07-23 00:00:00.


In [6]:
gold_bond.schedule

[Timestamp('2024-07-23 00:00:00'),
 Timestamp('2025-07-23 00:00:00'),
 Timestamp('2026-07-23 00:00:00')]

In [7]:
print(f"{gold_bond.fixed_coupon * 100:.4f}% p.a.")

5.3715% p.a.


### TODO:
* Compounding frequency
* Effective yield
* Different CCY for Commodity and Bond